<a href="https://colab.research.google.com/github/ShravaniR2412/SASS/blob/main/GLAMEASE_MARKET_BASKET_ANALYSIS(Association_rules).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_filled_dataset.csv to cleaned_filled_dataset.csv


In [ ]:
import pandas as pd

# Load the uploaded file
df = pd.read_csv("cleaned_filled_dataset.csv")

# Show first few rows
df.head()


,Transaction_ID,0,1,2,3,4
0,T00001,Dear Klairs Supple Preparation Facial Toner,HADA LABO Gokujyun Ultimate Moisturizing Lotion,Elsheskin Night Rejuvenation,AVOSKIN Miraculous Retinol Toner 20mL,HADA LABO Tamagohada Mild Peeling Face Wash 50gr
1,T00002,HOLIKA HOLIKA Good Cera Ceramide Cream In Serum,Mediheal Milk Brightening Serum,Nature Republic Bee Venom Toner,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil
2,T00003,Smooth Pure Cleansing Foam,LACOCO Aloe Vera Soothing Mist,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil
3,T00004,Elsheskin Day Rejuvenation,A'pieu Deep Clean Cleansing Oil,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil
4,T00005,KIEHLS Turmeric & Cranberry Seed Energizing Ra...,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil,bio beauty lab acne treatment face oil


In [ ]:
# Replace blank strings or whitespace-only with NaN
df = df.replace(r'^\s*$', pd.NA, regex=True)

# Fill NaN or empty cells with default product
default_product = "bio beauty lab acne treatment face oil"
df_filled = df.fillna(default_product)


In [ ]:
df_filled.isna().sum()


,0
Transaction_ID,0
0,0
1,0
2,0
3,0
4,0


In [ ]:
# Confirm there are no missing values left
df_filled.isna().sum()


,0
Transaction_ID,0
0,0
1,0
2,0
3,0
4,0


In [ ]:
transactions = df.apply(lambda row: row.dropna().tolist(), axis=1)
transactions = transactions.tolist()


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)


In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df_encoded, min_support=0.005, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head()


,support,itemsets
18,0.8894,(bio beauty lab acne treatment face oil)
10,0.1418,(Mediheal Milk Brightening Serum)
11,0.1382,(Nature Republic Bee Venom Toner)
9,0.1364,(HOLIKA HOLIKA Good Cera Ceramide Cream In Serum)
14,0.1326,(Shiseido Future Solution LX Universal Defense E)


In [ ]:
#  then we will check ki threshold confidence>5% and lift>1.2
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)
rules = rules[(rules['confidence'] >= 0.5) & (rules['lift'] >= 1.2)]  # Filter strong rules
rules.sort_values(by='lift', ascending=False).head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
377,(HADA LABO Gokujyun Ultimate Moisturizing Loti...,"(Dear Klairs Supple Preparation Facial Toner, ...",0.0904,0.0944,0.0566,0.626106,6.632481,1.0,0.048066,2.422078,0.933627,0.441498,0.587131,0.612841
372,"(Dear Klairs Supple Preparation Facial Toner, ...",(HADA LABO Gokujyun Ultimate Moisturizing Loti...,0.0944,0.0904,0.0566,0.599576,6.632481,1.0,0.048066,2.271594,0.937751,0.441498,0.559780,0.612841
376,"(Elsheskin Night Rejuvenation, AVOSKIN Miracul...","(Dear Klairs Supple Preparation Facial Toner, ...",0.0920,0.0942,0.0566,0.615217,6.530970,1.0,0.047934,2.354056,0.932691,0.436728,0.575201,0.608033
373,"(Dear Klairs Supple Preparation Facial Toner, ...","(Elsheskin Night Rejuvenation, AVOSKIN Miracul...",0.0942,0.0920,0.0566,0.600849,6.530970,1.0,0.047934,2.274830,0.934956,0.436728,0.560407,0.608033
369,"(Dear Klairs Supple Preparation Facial Toner, ...",(HADA LABO Gokujyun Ultimate Moisturizing Lotion),0.0700,0.1240,0.0566,0.808571,6.520737,1.0,0.047920,4.576119,0.910369,0.411936,0.781474,0.632512


In [ ]:
def get_similar_recommendations(product, rules_df, top_n=5):
    recommendations = []
    seen_items = set()  # To avoid duplicates

    for _, row in rules_df.iterrows():
        if product in row['antecedents']:
            for item in row['consequents']:
                if item not in seen_items:
                    seen_items.add(item)
                    recommendations.append((
                        item,
                        round(row['confidence'], 2),
                        round(row['lift'], 2),
                        round(row['support'], 3)
                    ))

    recommendations = sorted(recommendations, key=lambda x: (x[1], x[2]), reverse=True)
    return recommendations[:top_n]


In [ ]:
selected_product = "Whitelab Exfoliating Toner"  # Example
recommendations = get_similar_recommendations(selected_product, rules)

if recommendations:
    print(f"\n🛍️ USER BOUGHT: '{selected_product}' — ALSO BOUGHT:")
    for idx, (item, conf, lift, supp) in enumerate(recommendations, 1):
        print(f"{idx}. {item} (Confidence: {conf}, Lift: {lift}, Support: {supp})")
else:
    print("❌ No recommendations found. Try another product or lower thresholds.")



🛍️ USER BOUGHT: 'Whitelab Exfoliating Toner' — ALSO BOUGHT:
1. WESTCARE Rose Hydrating Toner (Confidence: 0.75, Lift: 5.86, Support: 0.095)
2. Bio Renew Foamy Cleanser 100g (Confidence: 0.74, Lift: 5.81, Support: 0.094)
3. Erha Acne Clarifying Gel (Confidence: 0.74, Lift: 5.79, Support: 0.094)
4. bio beauty lab acne treatment face oil (Confidence: 0.57, Lift: 5.5, Support: 0.073)


In [ ]:
unique_items = set()
for t in transactions:
    unique_items.update(t)

print(sorted(unique_items))

num_unique_items = len(unique_items)
print(num_unique_items)


[' Banana Boat Ultra Protect Faces Sunscreen Lotion SPF 50', ' DEAR ME BEAUTY Single Active Face Serum 10% Cica + Watermelon Extract ', ' DEAR ME BEAUTY Single Active Face Serum Retinol + Blueberry Extract ', ' DERMALUZ Lucem Smothees Power Serum with Galactomyces ', ' ERHA 8 Moisturizer For Very Dry Skin', ' Erha ERHA X AQUA Re-fresh Hydrating Face Mist Rp 93.500Rp 78.540-16%(22)', ' HOLIKA HOLIKA Good Cera Ceramide Cream In Serum ', ' JARKEEN Vit C Booster Serum ', ' JELLYS Pure Face Serum', ' KIEHLS Nightly Refining Micro Peel Concentrate 4ml ', ' Scarlett Whitening Brightly Ever After Day Cream', ' VOTRE PEAU Niacinamide Booster Ampoule ', ' WARDAH Lightening Facial Serum ', '27% HERBORIST Rose Water (Air Mawar)', "A'pieu Deep Clean Cleansing Oil", "A'pieu Deep Clean Cleansing Oil ", "A'pieu Deep Clean Foam Cleanser ", "A'pieu Deep Clean Foam Cleanser (Pore)", "A'pieu Madecassoside Cleansing Foam ", "A'pieu Pure Block Water Proof Sun Cream Spf50+Pa+++ ", 'ACWELL Aquaseal Soothing T

In [ ]:
# transactions list mei kitne total set it will check that
unique_items = set()
for transaction in transactions:
    unique_items.update(transaction)

print(f"🔢 Total Unique Items: {len(unique_items)}")


🔢 Total Unique Items: 5535
